importacion de librerias


In [147]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [148]:
df = pd.read_csv('./data/2024_Accidentalidad.csv', delimiter=';')


In [149]:
df.sample(10)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
13654,2024S013215,13/04/2024,0:05:00,"CALL. SIRIO, 60",60,3,RETIRO,Choque contra obstáculo fijo,Despejado,Turismo,Pasajero,De 25 a 29 años,Mujer,1.0,Atención en urgencias sin posterior ingreso,443597.0,4473415.0,N,NaN
15712,2024S014670,26/04/2024,22:00:00,ALTO DEL RETIRO,39,18,VILLA DE VALLECAS,Colisión lateral,NaN,Turismo,Conductor,De 55 a 59 años,Hombre,NaN,NaN,448938.0,4468800.0,N,NaN
25682,2024S021519,06/07/2024,19:20:00,PLAZA. MARIANO DE CAVIA / AVDA. MEDITERRANEO,3,3,RETIRO,Caída,Despejado,Ciclomotor,Conductor,De 25 a 29 años,Mujer,6.0,Asistencia sanitaria inmediata en centro de sa...,442428.0,4473210.0,N,NaN
34797,2024S027063,26/09/2024,9:32:00,CALL. DOCTOR ESQUERDO / CALL. ASTROS,125,3,RETIRO,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 50 a 54 años,Mujer,14.0,Sin asistencia sanitaria,443181.0,4473627.0,N,NaN
10995,2024S010264,22/03/2024,7:45:00,"CALL. SANCHO DAVILA, 1",1,4,SALAMANCA,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,Desconocido,Desconocido,14.0,Sin asistencia sanitaria,443834.0,4475573.0,N,NaN
9143,2024S008513,09/03/2024,5:05:00,CALL. RAIMUNDO FERNANDEZ VILLAVERDE / CALL. OR...,50,7,CHAMBERÍ,Colisión fronto-lateral,Lluvia débil,Furgoneta,Conductor,De 45 a 49 años,Hombre,14.0,Sin asistencia sanitaria,441022.0,4477533.0,N,NaN
30831,2024S024739,26/08/2024,19:00:00,AVDA. ANDALUCIA / GTA. MALAGA,18,12,USERA,Colisión frontal,Despejado,Furgoneta,Pasajero,De 50 a 54 años,Hombre,14.0,Sin asistencia sanitaria,441030.0,4469663.0,N,NaN
36127,2024S027822,04/10/2024,16:15:00,"AUTOV. A-6, +00600E",+00600E,9,MONCLOA-ARAVACA,Alcance,Despejado,Turismo,Conductor,De 45 a 49 años,Mujer,14.0,Sin asistencia sanitaria,437519.0,4477875.0,N,NaN
35773,2024S027629,02/10/2024,13:50:00,"CALL. ARROYO DE LAS PILILLAS, 37",37,14,MORATALAZ,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 50 a 54 años,Mujer,14.0,Sin asistencia sanitaria,444694.0,4473067.0,N,NaN
22987,2024S019959,16/06/2024,21:50:00,"AVDA. OPORTO, 8",8,11,CARABANCHEL,Colisión lateral,Despejado,Turismo,Conductor,De 45 a 49 años,Mujer,14.0,Sin asistencia sanitaria,438914.0,4470764.0,N,NaN


Analisis del dataset


https://datos.madrid.es/FWProjects/egob/Catalogo/Seguridad/Ficheros/Estructura_ConjuntoDatos_Accidentesv2.pdf


Lo primero que nos damos cuenta cuando leemos el anterior documento es que el año del accidente no viene en la fecha, si no que viene en los 4 primeros char del numero del expediente.


**localizacion** y **numero**: con la localizacion y el numero (numero de la calle) nos parece interesante quedarnosla, pero solo para la parte de entendimiento de los datos del cluster, ya que para el claster es mas util las coordenadas utm


para el codigo del distrito nos va a servir para hacer el cluster, pero como el numero en si no nos dice nada, pues entonces lo que vamos a hacer un mapeo del distrito para entender y tener la correlacion


para el tipo de accidente y para el estado metereologico haremos un mapeo simple de las 13 posibles variables (mapping)


en tipo de vehiculo habia demasiados valroes unicos (33) por lo que hemos intentado generalizar un poco para poder tratar con esta columna, haciendo un mapeo y un posible posterior onehotencoding


para el **rango de edad** vamos


Entendimiento del Dataset


Limpieza


Explicacion de las acciones realizadas


In [150]:
len(df['tipo_vehiculo'].unique())


33

In [151]:
df['tipo_vehiculo'].value_counts()

tipo_vehiculo
Turismo                              27406
Motocicleta hasta 125cc               2668
Furgoneta                             2667
Motocicleta > 125cc                   1469
Autobús                               1264
Camión rígido                          973
Todo terreno                           706
Bicicleta                              670
VMU eléctrico                          665
Ciclomotor                             387
Bicicleta EPAC (pedaleo asistido)      211
Tractocamión                           187
Maquinaria de obras                    151
Otros vehículos con motor              131
Vehículo articulado                     70
Autobús articulado                      69
Sin especificar                         40
Cuadriciclo ligero                      32
Autobus EMT                             17
Ciclo                                   13
Cuadriciclo no ligero                   12
Autocaravana                            11
Otros vehículos sin motor               

In [152]:
map_vehiculos = {
    'Turismo': 'Turismos y vehículos ligeros',
    'Todo terreno': 'Turismos y vehículos ligeros',
    'Autocaravana': 'Turismos y vehículos ligeros',
    'Motocicleta hasta 125cc': 'Motocicletas y ciclomotores',
    'Motocicleta > 125cc': 'Motocicletas y ciclomotores',
    'Ciclomotor': 'Motocicletas y ciclomotores',
    'Bicicleta EPAC (pedaleo asistido)': 'Motocicletas y ciclomotores',
    'Ciclomotor de tres ruedas': 'Motocicletas y ciclomotores',
    'Moto de tres ruedas hasta 125cc': 'Motocicletas y ciclomotores',
    'Moto de tres ruedas > 125cc': 'Motocicletas y ciclomotores',
    'Ciclo de motor L1e-A': 'Motocicletas y ciclomotores',
    'Ciclomotor de dos ruedas L1e-B': 'Motocicletas y ciclomotores',
    'Furgoneta': 'Vehículos de carga',
    'Camión rígido': 'Vehículos de carga',
    'Tractocamión': 'Vehículos de carga',
    'Vehículo articulado': 'Vehículos de carga',
    'Semiremolque': 'Vehículos de carga',
    'Remolque': 'Vehículos de carga',
    'Autobús': 'Vehículos de transporte público',
    'Autobús articulado': 'Vehículos de transporte público',
    'Autobus EMT': 'Vehículos de transporte público',
    'Bicicleta': 'Bicicletas y ciclos no motorizados',
    'Ciclo': 'Bicicletas y ciclos no motorizados',
    'Patinete no eléctrico': 'Bicicletas y ciclos no motorizados',
    'VMU eléctrico': 'Vehículos eléctricos ligeros',
    'Maquinaria de obras': 'Vehículos especializados',
    'Ambulancia SAMUR': 'Vehículos especializados',
    'Otros vehículos con motor': 'Vehículos especializados',
    'Otros vehículos sin motor': 'Vehículos especializados',
    'Sin especificar': 'Sin clasificar/especificar',
    'Cuadriciclo ligero': 'Sin clasificar/especificar',
    'Cuadriciclo no ligero': 'Sin clasificar/especificar'
}

In [153]:
df['tipo_vehiculo'] = df['tipo_vehiculo'].map(map_vehiculos)

Mapeo de rango de edades


In [154]:
map_edad = {
    'Desconocido': -1,
    'Menor de 5 años': 0,
    'De 6 a 9 años': 0,
    'De 10 a 14 años': 10,
    'De 15 a 17 años': 10,
    'De 18 a 20 años': 18,
    'De 21 a 24 años': 18,
    'De 25 a 29 años': 25,
    'De 30 a 34 años': 25,
    'De 35 a 39 años': 25,
    'De 40 a 44 años': 40,
    'De 45 a 49 años': 40,
    'De 50 a 54 años': 40,
    'De 55 a 59 años': 40,
    'De 60 a 64 años': 60,
    'De 65 a 69 años': 60,
    'De 70 a 74 años': 60,
    'Más de 74 años': 60
}

In [155]:
df['rango_edad'] = df['rango_edad'].map(map_edad)

Mapeo de estado meteorológico


In [156]:
df = df.fillna({
    'estado_meteorológico': 'Se desconoce'
})

print(df['estado_meteorológico'].unique())

map_est_met = {
    'Se Desconoce': -1,
    'Despejado': 0,
    'Nublado': 1,
    'Lluvia débil': 2,
    'LLuvia intensa': 3,
    'Granizando': 4,
    'Nevando': 5
}

df['estado_meteorológico'] = df['estado_meteorológico'].map(map_est_met)

['Lluvia débil' 'Despejado' 'Se desconoce' 'Nublado' 'LLuvia intensa'
 'Granizando' 'Nevando']


Mapeo Lesividad


In [157]:
print(df['lesividad'].unique())

['Ingreso inferior o igual a 24 horas' 'Sin asistencia sanitaria'
 'Asistencia sanitaria sólo en el lugar del accidente' nan
 'Atención en urgencias sin posterior ingreso'
 'Ingreso superior a 24 horas'
 'Asistencia sanitaria inmediata en centro de salud o mutua'
 'Asistencia sanitaria ambulatoria con posterioridad' 'Fallecido 24 horas']


In [158]:
df = df.fillna({
    'lesividad': 'Sin asistencia sanitaria',
    'cod_lesividad': 0
})

map_lesividad = {
    'Sin asistencia sanitaria': 'NONE',
    'Atención en urgencias sin posterior ingreso': 'LEVE',
    'Ingreso inferior o igual a 24 horas': 'LEVE',
    'Asistencia sanitaria ambulatoria con posterioridad': 'LEVE',
    'Asistencia sanitaria inmediata en centro de salud o mutua': 'LEVE',
    'Asistencia sanitaria sólo en el lugar del accidente': 'LEVE',
    'Ingreso inferior o igual a 24 horas': 'GRAVE',
    'Fallecido 24 horas': 'FALLECIDO'
}

In [159]:
df['lesividad'] = df['lesividad'].map(map_lesividad)

In [160]:
df['lesividad'].value_counts()

lesividad
NONE         30938
LEVE          7234
GRAVE         1494
FALLECIDO       24
Name: count, dtype: int64

Mapeo Alcohol y drogas


In [161]:
df = df.fillna({
    'positiva_droga': 0
})

map_alcohol = {
    'N': 0,
    'S': 1
}

df['positiva_alcohol'] = df['positiva_alcohol'].map(map_alcohol)

In [162]:
print('valores de positivos en alcohol: \n {} \n valores de positivos en droga: \n {}'.format(
    df['positiva_alcohol'].value_counts(),df['positiva_droga'].value_counts()
    ))

valores de positivos en alcohol: 
 positiva_alcohol
0.0    38878
1.0     1149
Name: count, dtype: int64 
 valores de positivos en droga: 
 positiva_droga
0.0    39995
1.0      170
Name: count, dtype: int64


In [163]:
df.head(10)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,2.0,Motocicletas y ciclomotores,Conductor,40,Hombre,2.0,GRAVE,444913.0,4481427.0,0.0,0.0
1,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,2.0,Turismos y vehículos ligeros,Conductor,40,Mujer,14.0,NONE,444913.0,4481427.0,0.0,0.0
2,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,2.0,Bicicletas y ciclos no motorizados,Conductor,25,Hombre,7.0,LEVE,440123.0,4475170.0,0.0,0.0
3,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,2.0,Motocicletas y ciclomotores,Conductor,25,Hombre,14.0,NONE,440123.0,4475170.0,0.0,0.0
4,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,0.0,Turismos y vehículos ligeros,Conductor,25,Hombre,0.0,NONE,440137.0,4475721.0,0.0,0.0
5,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,0.0,Turismos y vehículos ligeros,Conductor,40,Hombre,0.0,NONE,440137.0,4475721.0,0.0,0.0
6,2023S040312,25/02/2024,17:20:00,"CALL. PINOS ALTA, 93",93,6,TETUÁN,Colisión lateral,0.0,Vehículos de carga,Conductor,25,Hombre,14.0,NONE,440709.0,4480102.0,0.0,0.0
7,2023S040312,25/02/2024,17:20:00,"CALL. PINOS ALTA, 93",93,6,TETUÁN,Colisión lateral,0.0,Vehículos especializados,Conductor,40,Hombre,14.0,NONE,440709.0,4480102.0,0.0,0.0
8,2023S040316,07/03/2024,14:25:00,CTRA. BARRIO DE LA FORTUNA / CALL. PINAR DE SA...,14,10,LATINA,Alcance,2.0,Vehículos de carga,Conductor,60,Hombre,0.0,NONE,433983.0,4468473.0,0.0,0.0
9,2023S040316,07/03/2024,14:25:00,CTRA. BARRIO DE LA FORTUNA / CALL. PINAR DE SA...,14,10,LATINA,Alcance,2.0,Turismos y vehículos ligeros,Conductor,25,Hombre,0.0,NONE,433983.0,4468473.0,0.0,0.0


In [164]:
df['rango_edad'].value_counts()

rango_edad
 40    14519
 25    11600
 60     4485
-1      4316
 18     3895
 10      723
 0       627
Name: count, dtype: int64

In [165]:
df.columns

Index(['num_expediente', 'fecha', 'hora', 'localizacion', 'numero',
       'cod_distrito', 'distrito', 'tipo_accidente', 'estado_meteorológico',
       'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'sexo', 'cod_lesividad',
       'lesividad', 'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol',
       'positiva_droga'],
      dtype='object')

In [166]:
df['cod_lesividad'].unique()

array([ 2., 14.,  7.,  0.,  1.,  3.,  6.,  5.,  4.])

In [167]:
df['cod_lesividad'] = df['cod_lesividad'].astype(int)

In [168]:
df['year'] = df['num_expediente'].astype(str).str[:4].astype(int)
df['id'] = df['num_expediente'].astype(str).str[-6:]

In [169]:
df['year'].value_counts()

year
2024    40154
2023       11
Name: count, dtype: int64

In [170]:
df = pd.get_dummies(df, columns=['tipo_persona'])

In [171]:
df.head(10)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,...,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,year,id,tipo_persona_Conductor,tipo_persona_Pasajero,tipo_persona_Peatón
0,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,2.0,Motocicletas y ciclomotores,...,GRAVE,444913.0,4481427.0,0.0,0.0,2023,040280,True,False,False
1,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,2.0,Turismos y vehículos ligeros,...,NONE,444913.0,4481427.0,0.0,0.0,2023,040280,True,False,False
2,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,2.0,Bicicletas y ciclos no motorizados,...,LEVE,440123.0,4475170.0,0.0,0.0,2023,040309,True,False,False
3,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,2.0,Motocicletas y ciclomotores,...,NONE,440123.0,4475170.0,0.0,0.0,2023,040309,True,False,False
4,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,0.0,Turismos y vehículos ligeros,...,NONE,440137.0,4475721.0,0.0,0.0,2023,040310,True,False,False
5,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,0.0,Turismos y vehículos ligeros,...,NONE,440137.0,4475721.0,0.0,0.0,2023,040310,True,False,False
6,2023S040312,25/02/2024,17:20:00,"CALL. PINOS ALTA, 93",93,6,TETUÁN,Colisión lateral,0.0,Vehículos de carga,...,NONE,440709.0,4480102.0,0.0,0.0,2023,040312,True,False,False
7,2023S040312,25/02/2024,17:20:00,"CALL. PINOS ALTA, 93",93,6,TETUÁN,Colisión lateral,0.0,Vehículos especializados,...,NONE,440709.0,4480102.0,0.0,0.0,2023,040312,True,False,False
8,2023S040316,07/03/2024,14:25:00,CTRA. BARRIO DE LA FORTUNA / CALL. PINAR DE SA...,14,10,LATINA,Alcance,2.0,Vehículos de carga,...,NONE,433983.0,4468473.0,0.0,0.0,2023,040316,True,False,False
9,2023S040316,07/03/2024,14:25:00,CTRA. BARRIO DE LA FORTUNA / CALL. PINAR DE SA...,14,10,LATINA,Alcance,2.0,Turismos y vehículos ligeros,...,NONE,433983.0,4468473.0,0.0,0.0,2023,040316,True,False,False


In [172]:
df.columns

Index(['num_expediente', 'fecha', 'hora', 'localizacion', 'numero',
       'cod_distrito', 'distrito', 'tipo_accidente', 'estado_meteorológico',
       'tipo_vehiculo', 'rango_edad', 'sexo', 'cod_lesividad', 'lesividad',
       'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol',
       'positiva_droga', 'year', 'id', 'tipo_persona_Conductor',
       'tipo_persona_Pasajero', 'tipo_persona_Peatón'],
      dtype='object')

In [173]:
mapeo_accidentes = {
    'NONE':0,
    'LEVE':1,
    'GRAVE':2,
    'FALLECIDO':3
}

df['lesividad'] = df['lesividad'].map(mapeo_accidentes)

In [174]:
map_tipo_accidente = {
    'Colisión fronto-lateral': 'Colision doble',
    'Alcance': 'Alcance',
    'Colisión lateral': 'Colision doble',
    'Choque contra obstáculo fijo':'Choque',
    'Atropello a persona': 'Atropello a persona',
    'Caída':'Caida',
    'Colisión múltiple':'Colision multiple',
    'Colisión frontal':'Colision doble',
    'Otro':'Otras Causas',
    'Solo salida de la vía': 'Otras Causas',
    'Vuelco': 'Vuelco',
    'Atropello a animal': 'Otras Causas'
}

df['tipo_accidente'] = df['tipo_accidente'].map(map_tipo_accidente)

In [ ]:
df['hora'] = df['hora'].str.split(':').str[0].astype(int)
df['hora'].value_counts()

In [175]:
df_entrenamiento = df.drop(columns=['num_expediente', 'localizacion', 'numero', 'distrito', 'lesividad'], axis=1)

In [176]:
df_entrenamiento.head(10)

,fecha,hora,cod_distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,rango_edad,sexo,cod_lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,year,id,tipo_persona_Conductor,tipo_persona_Pasajero,tipo_persona_Peatón
0,04/01/2024,14:09:00,16,Colision doble,2.0,Motocicletas y ciclomotores,40,Hombre,2,444913.0,4481427.0,0.0,0.0,2023,040280,True,False,False
1,04/01/2024,14:09:00,16,Colision doble,2.0,Turismos y vehículos ligeros,40,Mujer,14,444913.0,4481427.0,0.0,0.0,2023,040280,True,False,False
2,15/02/2024,14:05:00,1,Colision doble,2.0,Bicicletas y ciclos no motorizados,25,Hombre,7,440123.0,4475170.0,0.0,0.0,2023,040309,True,False,False
3,15/02/2024,14:05:00,1,Colision doble,2.0,Motocicletas y ciclomotores,25,Hombre,14,440123.0,4475170.0,0.0,0.0,2023,040309,True,False,False
4,18/02/2024,10:40:00,7,Colision doble,0.0,Turismos y vehículos ligeros,25,Hombre,0,440137.0,4475721.0,0.0,0.0,2023,040310,True,False,False
5,18/02/2024,10:40:00,7,Colision doble,0.0,Turismos y vehículos ligeros,40,Hombre,0,440137.0,4475721.0,0.0,0.0,2023,040310,True,False,False
6,25/02/2024,17:20:00,6,Colision doble,0.0,Vehículos de carga,25,Hombre,14,440709.0,4480102.0,0.0,0.0,2023,040312,True,False,False
7,25/02/2024,17:20:00,6,Colision doble,0.0,Vehículos especializados,40,Hombre,14,440709.0,4480102.0,0.0,0.0,2023,040312,True,False,False
8,07/03/2024,14:25:00,10,Alcance,2.0,Vehículos de carga,60,Hombre,0,433983.0,4468473.0,0.0,0.0,2023,040316,True,False,False
9,07/03/2024,14:25:00,10,Alcance,2.0,Turismos y vehículos ligeros,25,Hombre,0,433983.0,4468473.0,0.0,0.0,2023,040316,True,False,False


Group by por id de accidente:

-   Hora a solo la hora
-   Rango de edad a sumatorio de edades
-   Tipo vehiculo one hot encoding y suma
-   Tipo de persona one hot encoding y suma
-   Sexo one hot encoding y suma
-   Positivo en drogas y alcohol sumar el numero de positivos
-   Lesividad la mas alta
